In [341]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [342]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [343]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 32

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-1.6393e+00, -1.3698e-01,  2.0613e-01,  9.5622e-01, -5.0885e-01,
          2.2161e+00, -1.2682e+00,  4.4251e-01,  3.4415e-01, -2.1501e-01,
          6.4293e-01,  1.7758e-01, -9.4408e-01, -8.9962e-01, -2.0110e-01,
         -7.6388e-01, -9.4343e-02, -5.4362e-01,  1.3712e+00,  6.4993e-01,
         -2.6553e-01, -8.0889e-01, -1.9406e-01,  1.3538e+00,  9.2374e-03,
          6.1935e-01,  4.9981e-02, -7.6900e-01,  2.2033e-01, -1.9121e-01,
         -2.0004e-01, -5.4597e-01],
        [-9.9945e-01,  1.8291e-01, -2.0589e+00,  6.4871e-01, -9.9940e-01,
         -7.6876e-01, -1.2826e-03,  1.0434e-01, -1.7798e-01,  3.1480e-01,
         -2.7057e-01, -3.5115e-01,  6.3346e-01, -6.4546e-01, -1.9132e-01,
          9.7976e-01, -6.4818e-01, -5.0489e-01, -1.2344e+00,  2.6901e-01,
         -2.0398e+00, -3.9326e-01, -3.4149e-01, -5.4871e-01, -1.3586e-01,
         -2.2778e+00,  5.7306e-01, -3.4074e-01,  7.6871e-02, -7.9640e-01,
          5.8957e-01,  1.3820e+00]], grad_fn=<EmbeddingBackward0>)


In [344]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [345]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 9,  2,  0,  9,  1,  1,  1, 10, 10, 10],
        [ 6,  1, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 3,  7,  7,  7,  1,  2,  9,  5, 10, 10],
        [ 1,  1,  8,  2,  4,  7,  0,  0,  4, 10],
        [ 9,  2,  0,  7,  4,  3,  6,  8,  8,  1],
        [ 0,  9,  0,  6,  3, 10, 10, 10, 10, 10],
        [ 5,  6,  9,  3,  5,  1,  5,  5,  6,  2],
        [ 5,  7,  1,  0,  6,  5, 10, 10, 10, 10],
        [ 9,  6,  8, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  5, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  3,  9, 10, 10, 10, 10, 10, 10, 10],
        [ 3,  0,  6,  7,  3,  1,  5,  5,  3, 10],
        [ 3,  2,  2,  2,  9,  3, 10, 10, 10, 10],
        [ 6,  1,  6, 10, 10, 10, 10, 10, 10, 10],
        [ 4,  7,  8,  5,  6,  4,  7,  8,  8,  7],
        [ 1,  7,  7,  0,  5,  8,  4,  6,  2,  1],
        [ 9,  5,  0, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  9,  4,  8,  9,  8,  7, 10, 10, 10],
        [ 6,  5,  7,  2,  4,  1,  7, 10, 10, 10],
        [ 1,  4,  2,  2,  5,  2, 10, 10, 10, 10],


In [346]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [347]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [348]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 322.05it/s]

[3.7837975563049318, 2.2831413497924804, 2.0418106884002687, 1.788407714653015, 1.9473170288085937, 1.5431591753005982, 1.3738824974060058, 1.4367087995529175, 1.469084016418457, 1.192953130531311]
[0.0036, 0.0129, 0.0174, 0.0235, 0.0205, 0.0291, 0.0358, 0.0354, 0.0333, 0.0404]
[0.5032020872865275, 0.5571373850868233, 0.5655516688529368, 0.5754043421244354, 0.5702112163146394, 0.5831360192335713, 0.5899178832116788, 0.5890527044758711, 0.5885127737226278, 0.5966522087767896]

[2.334340009689331, 2.7630862188339234, 1.5832197151184082, 3.085221704483032, 2.653798294067383, 2.03323722076416, 1.6169755506515502, 1.3904020366668701, 1.6502924098968506, 1.615731296300888]
[0.009, 0.011, 0.037, 0.01, 0.009, 0.02, 0.026, 0.036, 0.035, 0.041]
[0.5493782004389174, 0.5272494513533285, 0.5715069495245062, 0.5243233357717629, 0.5314557425018288, 0.5704096561814191, 0.5827542062911485, 0.583394294074616, 0.5828456474030724, 0.5885149963423555]
